In [1]:
import os
os.chdir('../')

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCustomCNNConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_classes: int


In [3]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_custom_cnn_config(self):
        config = self.config.prepare_custom_cnn
        
        create_directories([config.root_dir])

        prepare_custom_cnn_config = {
            'IMAGE_SIZE': self.params.IMAGE_SIZE,
            'CLASSES': self.params.CLASSES,
            'LEARNING_RATE': self.params.LEARNING_RATE,
            'BASE_MODEL_PATH': Path(config.base_model_path),
            'UPDATED_BASE_MODEL_PATH': Path(config.updated_base_model_path)
        }

        return prepare_custom_cnn_config


In [5]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

[2023-08-27 01:34:02,072: INFO: utils: Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.]
[2023-08-27 01:34:02,072: INFO: utils: NumExpr defaulting to 8 threads.]


In [6]:
import tensorflow as tf
from pathlib import Path

class PrepareCustomCNN:
    def __init__(self, config):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.Sequential([
            tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=tuple(self.config['IMAGE_SIZE'])),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Conv2D(512, (3, 3), activation='relu'),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(512, activation='relu'),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(self.config['CLASSES'], activation='softmax')
        ])

        self.save_model(path=self.config['BASE_MODEL_PATH'], model=self.model)

    @staticmethod
    def _prepare_full_model(model, learning_rate):
        model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        model.summary()
        return model

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            learning_rate=self.config['LEARNING_RATE']
        )
        self.save_model(path=self.config['UPDATED_BASE_MODEL_PATH'], model=self.full_model)

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


In [7]:
try:
    config = ConfigurationManager()  
    prepare_custom_cnn_config = config.get_prepare_custom_cnn_config()
    prepare_custom_cnn = PrepareCustomCNN(config=prepare_custom_cnn_config)
    
    prepare_custom_cnn.get_base_model()
    prepare_custom_cnn.update_base_model()
except Exception as e:
    raise e


[2023-08-27 01:34:02,521: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-27 01:34:02,523: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-27 01:34:02,525: INFO: common: created directory at: artifacts]
[2023-08-27 01:34:02,526: INFO: common: created directory at: artifacts/prepare_base_model]
[2023-08-27 01:34:03,240: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 64)     0         
 )                                                               
                                           